<font size="5">EEG-Data-Set Creation</font>

In [1]:
# Import modules and dependencies

import os
from os import listdir
from os.path import isdir, join
import pathlib
import shutil
import scipy
from scipy import signal
import sys

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras

import random
import mne
import pyedflib
from collections import OrderedDict
from itertools import repeat

import pandas as pd

In [2]:
# parameter file constants:
# these are reserved words used to parse paramter files
# Source: https://isip.piconepress.com/projects/tuh_eeg/downloads/nedc_pystream/
KEYWORD_VERSION = "version"
KEYWORD_VERSION_NUMBER = "1.0"
KEYWORD_HELP = "-help"

KEYWORD_CSEL = "channel_selection"
KEYWORD_MONTAGE = "montage"
KEYWORD_MMODE = "match_mode"

KEYWORD_NULL = "(null)"
KEYWORD_EXACT = "exact"
KEYWORD_PARTIAL = "partial"

In [3]:
# parameter file constants:
# these are reserved symbols used to parse paramter files
# Source: https://isip.piconepress.com/projects/tuh_eeg/downloads/nedc_pystream/
DELIM_BCLOSE = '}'
DELIM_BOPEN = '{'
DELIM_CLOSE = ']'
DELIM_COLON = ':'
DELIM_COMMA = ','
DELIM_COMMENT = '#'
DELIM_EQUAL = '='
DELIM_NEWLINE = '\n'
DELIM_NULL = ''
DELIM_QUOTE = '"'
DELIM_SPACE = ' '
DELIM_SUB = "--"

In [4]:
# If needed, set seed for experiment reproducibility

seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

<font size="5">1. Load Data-Set</font>

In [5]:
# Define Directory

train_dir = pathlib.Path('data/edf/train')
val_dir = pathlib.Path('data/edf/eval')

In [6]:
# Create array with target classes
# In this data set there are only 2 classes

targets = ['normal', 'abnormal']

print('Classes in TUH EEG abnormal Corpus: ', targets)

Classes in TUH EEG abnormal Corpus:  ['normal', 'abnormal']


In [7]:
# Create a list with all training files

filenames_train = []
for root, dirs, files in os.walk(train_dir):
    for file in files:
        if file.endswith('.edf'):
            filenames_train.append(join(str(root),str(file)))


In [8]:
# Check filenames_train

print(filenames_train[1])

data/edf/train/normal/01_tcp_ar/021/00002128/s002_2012_04_12/00002128_s002_t000.edf


In [9]:
# Create y_train, source readme: There are 1371 normal and 1346 abnormal eeg files

y_train = []

y_train.extend([0 for i in range(1371)])
y_train.extend([1 for i in range(1346)])

In [10]:
# Are x and y same length? 

print(len(filenames_train))
print(len(y_train))

2717
2717


In [11]:
# Create a list with all validation files

filenames_val = []
for root, dirs, files in os.walk(val_dir):
    for file in files:
        if file.endswith('.edf'):
            filenames_val.append(join(str(root),str(file)))

In [12]:
# Create y_val, source readme: There are 150 normal and 126 abnormal eeg files

y_val = []

y_val.extend([0 for i in range(150)])
y_val.extend([1 for i in range(126)])

In [13]:
# Are x and y same length? 

print(len(filenames_val))
print(len(y_val))

276
276


In [14]:
# Append training and validation files

filenames = [*filenames_train,*filenames_val]
y = [*y_train,*y_val]

In [15]:
# Check the length

print(len(filenames))
print(len(y))

2993
2993


In [16]:
# Check filenames

print(filenames[0])
print(filenames[2717])
print(filenames[2718])
print(filenames_val[0])

data/edf/train/normal/01_tcp_ar/029/00002972/s002_2013_01_25/00002972_s002_t000.edf
data/edf/eval/normal/01_tcp_ar/029/00002940/s003_2009_10_22/00002940_s003_t001.edf
data/edf/eval/normal/01_tcp_ar/065/00006598/s001_2010_03_22/00006598_s001_t000.edf
data/edf/eval/normal/01_tcp_ar/029/00002940/s003_2009_10_22/00002940_s003_t001.edf


<font size="5">Attention</font>

The following functions are copied from the official code from TUH EEG (https://isip.piconepress.com/projects/tuh_eeg/downloads/nedc_pystream/). There were some errors when importing this code diretly from the website. Therefor, needed functions were simply copied into this notebook.

In [17]:
# function: nedc_load_edf
#
# arguments: 
#   fname: filename (input)
#
# return: 
#   labels: store the EDF signal labels
#   fsamp: store the EDF signal sample frequency
#   sig: signals in the EDF file
#
# this function loads the EDF and return the signals
#
def nedc_load_edf(fname_a):

    # open an EDF file 
    #
    try:
        fp = pyedflib.EdfReader(fname_a)
    except IOError:
        print("%s (%s: %s): failed to open %s" %
            (sys.argv[0], __name__, "nedc_load_edf", fname_a))
        exit(-1)

    # get the metadata that we need:
    #  convert the labels to ascii and remove whitespace 
    #  to make matching easier
    #
    num_chans = fp.signals_in_file
    labels_tmp = fp.getSignalLabels()
    labels = [str(lbl.replace(' ', '')) for lbl in labels_tmp]

    # load each channel
    #
    sig = []
    fsamp = []
    for i in range(num_chans):
        sig.append(fp.readSignal(i))
        fsamp.append(fp.getSampleFrequency(i))

    # exit gracefully
    #
    return (fsamp, sig, labels)
#
# end of function 

In [18]:
# function: nedc_select_channels
#
# arguments:
#   params: parameter block dictionary
#   fsamp: the sample frequencies for each channel
#   sig: the signal data
#   labels: the channel labels
#
# return:
#   fsamp_sel: output sample frequency list
#   sig_sel: output signal data
#   labels_sel: output channel labels
#
# This function returns selects channels from a signal and returns a
# subset of the channels.
#
def nedc_select_channels(params_a, fsamp_a, sig_a, labels_a):
    
    # declare local variables
    #
    fsamp_sel = []
    sig_sel = []
    labels_sel = []

    # extract the list of channels from the parameter block
    #
    chan_list = params_a.get(KEYWORD_CSEL).split(DELIM_COMMA)
    
    # if the channel list contains null, simply copy the input to the output
    #
    if KEYWORD_NULL in chan_list:
        return (fsamp_a, sig_a, labels_a)

    # else: copy selected channels
    #
    for lbl in chan_list:

        # look up the label in the original signal
        #
        pos = nedc_get_pos(lbl, labels_a, params_a[KEYWORD_MMODE])

        # append the corresponding signal
        #
        if pos >= int(0):
            fsamp_sel.append(fsamp_a[pos])
            sig_sel.append(sig_a[pos])
            labels_sel.append(labels_a[pos])
        else:
            print("%s (%s: %s): failed to find label %s" %
                (sys.argv[0], __name__, "nedc_select_channels", lbl))
            exit(-1)

    # exit gracefully
    #
    return fsamp_sel, sig_sel, labels_sel
#
# end of function 

In [19]:
# function: nedc_parse_montage
#
# arguments:
#   params: parameter block dictionary
#
# return:
#   montage: a list of n-tuples containing a montage specification
#
# This function converts the montage in the parameter block to a more
# user-friendly data structure.
#
def nedc_parse_montage(params_a):

     # loop over all montage entries:
     #  build a list that contains the channel index, the output label,
     #  the first input channel, and the second channel if a difference
     #  is specified.
     #
     montage = []

     for str in params_a[KEYWORD_MONTAGE]:

          # split the line into two pieces: the channel index
          # and the labels
          #
          parts = str.split(DELIM_COMMA)          

          # split the right-hand side into output label and input label
          #
          subparts = parts[1].split(DELIM_COLON)

          # split the input label into two terms if a difference is specified
          #
          expparts = subparts[1].split(DELIM_SUB)

          # assemble it into a full list
          #
          parts[1] = subparts[0]
          parts.append(expparts[0])
          if len(expparts) > 1:
               parts.append(expparts[1])
          else:
               parts.append(KEYWORD_NULL)
          montage.append(parts)

     # exit gracefully
     #
     return montage
#
# end of function

In [20]:
# function: nedc_apply_montage
#
# arguments:
#   params: parameter block dictionary
#   fsamp: the sample frequencies for each channel
#   sig: the signal data
#   labels: the channel labels
#
# return:
#   fsamp_mont: output sample frequency list
#   sig_mont: output signal data
#   labels_mont: output channel labels
#
# This function applys a montage to a signal.
#
def nedc_apply_montage(params_a, fsamp_a, sig_a, labels_a):

     # initialize the output variables
     #
     fsamp_mont = []
     sig_mont = []
     labels_mont = []

     # if the montage specification contains null, simply copy the 
     # input to the output
     #
     if KEYWORD_NULL in params_a[KEYWORD_MONTAGE.lower()]:
        return (fsamp_a, sig_a, labels_a)

     # convert the raw format of the montage into something
     # that is easier to process
     #
     montage = nedc_parse_montage(params_a)

     # loop over the output montage
     #
     for i in range(len(montage)):

          # get the position of the first operand
          #
          pos1 = nedc_get_pos(montage[i][2], labels_a,
                              params_a[KEYWORD_MMODE])
          if montage[i][3] != KEYWORD_NULL:
               pos2 = nedc_get_pos(montage[i][3], labels_a,
                                   params_a[KEYWORD_MMODE])
          else:
               pos2 = int(-1)

          # compute the new length as the shorter of the two
          #
          min_len = len(sig_a[pos1])
          if (pos2 >= int(0)):
               if len(sig_a[pos2]) < min_len:
                    min_len = len(sig_a[pos2])

          # copy the first signal
          #
          sig_mont.append(sig_a[pos1])
          sig_mont[i] = sig_mont[i][:min_len]

          # difference the two signals if necessary
          #
          if pos2 >= int(0):
               for j in range(min_len):
                    sig_mont[i][j] -= sig_a[pos2][j]

          # append the metadata
          #
          fsamp_mont.append(fsamp_a[pos1])
          labels_mont.append(montage[i][1])
    
     # exit gracefully
     #
     return (fsamp_mont, sig_mont, labels_mont)
#
# end of function

In [21]:
# function: nedc_load_parameters
#
# arguments: 
#   pfile: parameter file
#
# return:
#   values: an ordered dictionary that contains the name/value pairs in
#           the parameter file
#
# This function loads a parameter file and returns the associated name/value
# pairs in a dictionary data structure. Note that the montage specification
# is stored as one entry (a list) in the dictionary.
#
def nedc_load_parameters(pfile_a):

     # declare local variables
     #
     values = OrderedDict()
     keyword_upcase = KEYWORD_MONTAGE.upper()

     # open the file
     #
     try:
          fp = open(pfile_a, "r")
     except:
          print("%s (%s: %s): file not found (%s)" %
                (sys.argv[0], __name__, "nedc_load_parameters", pfile_a))
          return None

     # loop over all lines in the file
     #
     flag_pblock = False
     flag_montage = False
     for line in fp:
          
          # clean up the line
          #
          str = line.replace(DELIM_SPACE, DELIM_NULL)
          str = str.replace(DELIM_NEWLINE, DELIM_NULL)

          # throw away commented and blank lines
          #
          if (str.startswith(DELIM_COMMENT) == True) or (len(str) == 0):
               pass

          # check for the version
          #
          elif str.startswith(KEYWORD_VERSION) == True:
               parts = str.split(DELIM_EQUAL)
               if parts[1] != KEYWORD_VERSION_NUMBER:
                    print("%s (%s: %s): incorrect version number (%s)" %
                          (sys.argv[0], __name__, "nedc_load_parameters",
                           parts[1]))
                    return None

          # check for the beginning of a parameter block
          #
          elif (str.startswith(keyword_upcase) == True) and \
               (DELIM_BOPEN in str):
               flag_pblock = True

          # check for the end of a parameter block:
          # note that we exit if we hit the end of the parameter block
          #
          elif (flag_pblock == True) and (DELIM_BCLOSE in str):
               fp.close();
               break;

          # otherwise, if the parameter block has started, decode a parameter
          # by splitting and assigning to a dictionary
          #
          elif (flag_pblock == True):
               parts = str.split(DELIM_EQUAL)

               # check for the first occurrence of a montage entry and
               # initialize a list
               #
               if (parts[0] == KEYWORD_MONTAGE) and (flag_montage == False):
                    values[parts[0]] = []
                    flag_montage = True
               
               # if it is a montage keyword: append the montage list
               #
               if (parts[0] == KEYWORD_MONTAGE):
                    values[parts[0]].append(parts[1].replace(
                         DELIM_QUOTE, DELIM_NULL))

               # else: treat it as a normal name/value pair
               #
               else:
                    values[parts[0]] = parts[1].replace(
                         DELIM_QUOTE, DELIM_NULL)

     # close the file pointer
     #
     fp.close()

     # make sure we found a block
     #
     if flag_pblock == False:
          print("%s (%s: %s): invalid parameter file (%s)" %
                (sys.argv[0], __name__, "nedc_load_parameters", pfile_a))
          return None

     # exit gracefully
     #
     return values
#
# end of function

In [22]:
# function: nedc_get_pos
#
# arguments:
#   lbl: label to be located
#   labels: list of labels
#   mmode: match mode
#
# return:
#   pos: the position in the list
#
# This function locates a label on the list and returns the position.
#
def nedc_get_pos(lbl_a, labels_a, mmode_a):

    # declare local variables
    #
    indices = []

    # mode: exact
    #  note that we return the first match
    #
    if mmode_a == KEYWORD_EXACT:
        pos = labels_a.index(lbl_a)
        if pos >= int(0):
            indices.append(pos)
        else:
            indices.append(int(-1))

    # mode: partial
    #
    else:
        indices = [i for i, elem in enumerate(labels_a) if lbl_a in elem]

    # exit gracefully
    #
    if len(indices) == 0:
        return int(-1)
    else:
        return indices[0]

<font size="5">Attention</font>

The copied functions end here.

In [23]:
#Load parameters for TCP montage

params_dir = pathlib.Path('params04.txt')
params = nedc_load_parameters(params_dir)

In [24]:
# Shuffle filenames and unzip

filenames_target = list(zip(filenames, y))
random.shuffle(filenames_target)
filenames, y = zip(*filenames_target)

<font size="5">2. Create Data-Set</font>

In [25]:
# Create data set

num=250*60
uint_std = 64
uint_mean = 127

def create_raw_data(path):
    
    fsamp, sig, labels = nedc_load_edf(path)
    
    # Apply montage to the signal
    
    samp_mont, sig_mont, labels_mont = nedc_apply_montage(params, fsamp, sig, labels)
    
    # Take only the first 60 seconds
    
    stop = samp_mont[0] * 60
    stop = int(stop)
    sig_mont = np.array(sig_mont)
    sig_mont = sig_mont[:,:stop]
    
    # Resample the data to 250Hz
    
    resampled_data = signal.resample(sig_mont, num = num, axis=-1)
    
    # Convert to z-score per eeg-session
    
    eeg_mean = np.mean(resampled_data)
    eeg_std = np.std(resampled_data)

    resampled_data = (resampled_data - eeg_mean) / eeg_std
    
    resampled_data = (resampled_data * uint_std) + uint_mean
    resampled_data = np.clip(resampled_data, 0, 255)
    resampled_data = resampled_data.astype('uint8')  
    
    
    # Reshape data into one second long matrices
    output = resampled_data
    
    return output

In [26]:
# Create Actual data set

def extract_features(samples, labels):
    prob_cnt = 0
    out_x = []
    out_y = []
    temp_y = 0
        
    for index, filename in enumerate(samples):

        # Create path from given filename and target item
        path = filename

        # Create MFCCs
        raw_data = create_raw_data(path)

        if raw_data.shape == (22,15000):
            out_x.append(raw_data)
            out_y.append(labels[index])
            print(index, filename, labels[index])
        else:
            print('Dropped:', index, raw_data.shape)
            prob_cnt += 1

    return out_x, out_y, prob_cnt

In [27]:
# Create train, and validation set

x, y, prob = extract_features(filenames, y)
print('Removed percentage:', prob / len(y))


0 data/edf/train/abnormal/01_tcp_ar/087/00008719/s002_2012_04_19/00008719_s002_t000.edf 1
1 data/edf/eval/normal/01_tcp_ar/025/00002508/s002_2009_11_13/00002508_s002_t001.edf 0
2 data/edf/train/normal/01_tcp_ar/108/00010816/s001_2013_10_03/00010816_s001_t001.edf 0
3 data/edf/train/normal/01_tcp_ar/101/00010165/s001_2013_01_16/00010165_s001_t000.edf 0
4 data/edf/train/abnormal/01_tcp_ar/064/00006482/s002_2013_06_17/00006482_s002_t000.edf 1
5 data/edf/train/abnormal/01_tcp_ar/043/00004392/s004_2013_02_21/00004392_s004_t000.edf 1
6 data/edf/train/normal/01_tcp_ar/098/00009850/s001_2013_04_09/00009850_s001_t000.edf 0
7 data/edf/train/normal/01_tcp_ar/107/00010733/s001_2013_09_12/00010733_s001_t000.edf 0
8 data/edf/train/abnormal/01_tcp_ar/088/00008890/s001_2012_04_23/00008890_s001_t000.edf 1
9 data/edf/eval/abnormal/01_tcp_ar/091/00009102/s001_2012_07_11/00009102_s001_t000.edf 1
10 data/edf/train/abnormal/01_tcp_ar/086/00008639/s001_2012_01_09/00008639_s001_t000.edf 1
11 data/edf/train/abn

92 data/edf/eval/normal/01_tcp_ar/078/00007852/s001_2011_06_02/00007852_s001_t000.edf 0
93 data/edf/train/normal/01_tcp_ar/094/00009405/s001_2012_09_25/00009405_s001_t000.edf 0
94 data/edf/eval/abnormal/01_tcp_ar/065/00006566/s001_2010_04_15/00006566_s001_t000.edf 1
95 data/edf/train/normal/01_tcp_ar/103/00010360/s001_2013_05_29/00010360_s001_t000.edf 0
96 data/edf/train/abnormal/01_tcp_ar/010/00001052/s008_2011_10_20/00001052_s008_t000.edf 1
97 data/edf/train/normal/01_tcp_ar/081/00008184/s003_2012_10_26/00008184_s003_t001.edf 0
98 data/edf/train/abnormal/01_tcp_ar/100/00010022/s005_2013_02_13/00010022_s005_t001.edf 1
99 data/edf/train/normal/01_tcp_ar/073/00007335/s001_2010_11_23/00007335_s001_t000.edf 0
100 data/edf/train/normal/01_tcp_ar/105/00010590/s002_2013_08_07/00010590_s002_t001.edf 0
101 data/edf/train/normal/01_tcp_ar/067/00006789/s001_2010_08_10/00006789_s001_t000.edf 0
102 data/edf/train/abnormal/01_tcp_ar/076/00007642/s001_2011_02_07/00007642_s001_t001.edf 1
103 data/edf

183 data/edf/eval/normal/01_tcp_ar/054/00005457/s001_2011_12_05/00005457_s001_t000.edf 0
184 data/edf/train/abnormal/01_tcp_ar/067/00006785/s002_2013_07_29/00006785_s002_t000.edf 1
185 data/edf/train/abnormal/01_tcp_ar/005/00000563/s003_2010_03_23/00000563_s003_t000.edf 1
186 data/edf/train/normal/01_tcp_ar/095/00009529/s001_2012_12_07/00009529_s001_t000.edf 0
187 data/edf/train/normal/01_tcp_ar/088/00008888/s001_2012_06_19/00008888_s001_t001.edf 0
188 data/edf/train/abnormal/01_tcp_ar/076/00007633/s001_2011_02_01/00007633_s001_t000.edf 1
189 data/edf/train/abnormal/01_tcp_ar/099/00009934/s001_2013_03_16/00009934_s001_t000.edf 1
190 data/edf/train/abnormal/01_tcp_ar/082/00008297/s002_2012_10_22/00008297_s002_t000.edf 1
191 data/edf/train/abnormal/01_tcp_ar/062/00006236/s001_2009_12_15/00006236_s001_t000.edf 1
192 data/edf/train/abnormal/01_tcp_ar/064/00006449/s001_2010_07_16/00006449_s001_t001.edf 1
193 data/edf/train/normal/01_tcp_ar/083/00008321/s001_2011_11_10/00008321_s001_t000.edf

273 data/edf/train/abnormal/01_tcp_ar/077/00007773/s008_2011_12_16/00007773_s008_t001.edf 1
274 data/edf/train/normal/01_tcp_ar/073/00007305/s003_2011_10_24/00007305_s003_t001.edf 0
275 data/edf/train/abnormal/01_tcp_ar/002/00000254/s005_2010_11_15/00000254_s005_t000.edf 1
276 data/edf/train/abnormal/01_tcp_ar/084/00008494/s001_2012_02_20/00008494_s001_t000.edf 1
277 data/edf/train/normal/01_tcp_ar/097/00009785/s001_2012_07_09/00009785_s001_t000.edf 0
278 data/edf/eval/normal/01_tcp_ar/103/00010344/s001_2013_05_17/00010344_s001_t000.edf 0
279 data/edf/train/abnormal/01_tcp_ar/104/00010431/s001_2013_06_17/00010431_s001_t000.edf 1
280 data/edf/train/abnormal/01_tcp_ar/099/00009915/s001_2013_04_24/00009915_s001_t000.edf 1
281 data/edf/train/normal/01_tcp_ar/069/00006984/s003_2013_09_05/00006984_s003_t000.edf 0
282 data/edf/train/abnormal/01_tcp_ar/066/00006648/s001_2010_03_25/00006648_s001_t000.edf 1
283 data/edf/train/normal/01_tcp_ar/086/00008676/s001_2012_02_10/00008676_s001_t002.edf 0

364 data/edf/eval/normal/01_tcp_ar/062/00006227/s001_2009_12_15/00006227_s001_t001.edf 0
365 data/edf/train/abnormal/01_tcp_ar/086/00008602/s001_2011_12_27/00008602_s001_t001.edf 1
366 data/edf/train/abnormal/01_tcp_ar/061/00006123/s002_2011_04_22/00006123_s002_t001.edf 1
367 data/edf/train/abnormal/01_tcp_ar/071/00007174/s001_2010_02_08/00007174_s001_t000.edf 1
368 data/edf/train/normal/01_tcp_ar/088/00008865/s001_2012_04_16/00008865_s001_t000.edf 0
369 data/edf/train/abnormal/01_tcp_ar/084/00008445/s001_2011_10_07/00008445_s001_t001.edf 1
370 data/edf/train/normal/01_tcp_ar/081/00008178/s001_2011_10_27/00008178_s001_t000.edf 0
371 data/edf/eval/normal/01_tcp_ar/061/00006156/s001_2009_09_23/00006156_s001_t000.edf 0
372 data/edf/train/normal/01_tcp_ar/100/00010063/s001_2013_02_11/00010063_s001_t000.edf 0
373 data/edf/train/abnormal/01_tcp_ar/096/00009649/s002_2012_12_06/00009649_s002_t000.edf 1
374 data/edf/train/normal/01_tcp_ar/004/00000407/s002_2011_08_29/00000407_s002_t001.edf 0
37

...

2620 data/edf/train/abnormal/01_tcp_ar/092/00009203/s001_2012_09_11/00009203_s001_t000.edf 1
2621 data/edf/eval/abnormal/01_tcp_ar/105/00010506/s001_2013_07_01/00010506_s001_t000.edf 1
2622 data/edf/train/abnormal/01_tcp_ar/099/00009935/s001_2013_03_10/00009935_s001_t001.edf 1
2623 data/edf/train/abnormal/01_tcp_ar/106/00010638/s001_2013_08_26/00010638_s001_t000.edf 1
2624 data/edf/train/normal/01_tcp_ar/104/00010445/s001_2013_06_18/00010445_s001_t000.edf 0
2625 data/edf/train/normal/01_tcp_ar/088/00008868/s001_2012_04_13/00008868_s001_t000.edf 0
2626 data/edf/train/abnormal/01_tcp_ar/061/00006171/s002_2012_12_06/00006171_s002_t000.edf 1
2627 data/edf/train/abnormal/01_tcp_ar/077/00007773/s004_2011_12_15/00007773_s004_t001.edf 1
2628 data/edf/train/abnormal/01_tcp_ar/085/00008552/s002_2013_04_19/00008552_s002_t000.edf 1
2629 data/edf/train/abnormal/01_tcp_ar/106/00010618/s001_2013_08_01/00010618_s001_t000.edf 1
2630 data/edf/train/abnormal/01_tcp_ar/094/00009482/s001_2012_10_26/0000948

2710 data/edf/train/abnormal/01_tcp_ar/018/00001826/s003_2012_03_23/00001826_s003_t000.edf 1
2711 data/edf/train/abnormal/01_tcp_ar/098/00009863/s001_2013_04_03/00009863_s001_t000.edf 1
2712 data/edf/train/abnormal/01_tcp_ar/076/00007625/s001_2011_01_17/00007625_s001_t000.edf 1
2713 data/edf/train/abnormal/01_tcp_ar/074/00007459/s001_2010_12_06/00007459_s001_t000.edf 1
2714 data/edf/train/abnormal/01_tcp_ar/079/00007946/s003_2013_03_18/00007946_s003_t000.edf 1
2715 data/edf/train/abnormal/01_tcp_ar/074/00007409/s002_2012_03_09/00007409_s002_t000.edf 1
2716 data/edf/train/normal/01_tcp_ar/074/00007430/s001_2010_10_15/00007430_s001_t001.edf 0
2717 data/edf/train/abnormal/01_tcp_ar/051/00005103/s003_2010_08_11/00005103_s003_t000.edf 1
2718 data/edf/train/abnormal/01_tcp_ar/065/00006546/s014_2011_03_15/00006546_s014_t000.edf 1
2719 data/edf/train/abnormal/01_tcp_ar/077/00007792/s001_2011_05_30/00007792_s001_t001.edf 1
2720 data/edf/train/normal/01_tcp_ar/096/00009653/s001_2012_11_05/000096

2800 data/edf/eval/normal/01_tcp_ar/074/00007481/s001_2010_12_03/00007481_s001_t001.edf 0
2801 data/edf/train/abnormal/01_tcp_ar/071/00007118/s001_2010_01_29/00007118_s001_t000.edf 1
2802 data/edf/train/normal/01_tcp_ar/104/00010422/s001_2013_06_17/00010422_s001_t002.edf 0
2803 data/edf/train/abnormal/01_tcp_ar/023/00002367/s004_2010_07_29/00002367_s004_t000.edf 1
2804 data/edf/train/abnormal/01_tcp_ar/104/00010469/s001_2013_06_27/00010469_s001_t000.edf 1
2805 data/edf/train/normal/01_tcp_ar/084/00008482/s002_2013_02_25/00008482_s002_t000.edf 0
2806 data/edf/train/abnormal/01_tcp_ar/087/00008719/s003_2012_09_10/00008719_s003_t000.edf 1
2807 data/edf/eval/abnormal/01_tcp_ar/077/00007726/s002_2011_06_02/00007726_s002_t000.edf 1
2808 data/edf/eval/normal/01_tcp_ar/013/00001355/s003_2009_12_08/00001355_s003_t000.edf 0
2809 data/edf/train/normal/01_tcp_ar/088/00008853/s001_2012_05_08/00008853_s001_t000.edf 0
2810 data/edf/train/normal/01_tcp_ar/080/00008061/s001_2011_03_29/00008061_s001_t00

2890 data/edf/train/abnormal/01_tcp_ar/093/00009347/s002_2013_03_12/00009347_s002_t000.edf 1
2891 data/edf/train/abnormal/01_tcp_ar/038/00003816/s002_2012_02_24/00003816_s002_t000.edf 1
2892 data/edf/eval/abnormal/01_tcp_ar/096/00009693/s002_2013_01_04/00009693_s002_t001.edf 1
2893 data/edf/train/abnormal/01_tcp_ar/090/00009067/s001_2012_04_09/00009067_s001_t002.edf 1
2894 data/edf/train/normal/01_tcp_ar/035/00003523/s004_2012_04_02/00003523_s004_t001.edf 0
2895 data/edf/train/abnormal/01_tcp_ar/060/00006059/s004_2012_06_25/00006059_s004_t000.edf 1
2896 data/edf/train/abnormal/01_tcp_ar/088/00008823/s002_2013_05_28/00008823_s002_t000.edf 1
2897 data/edf/train/normal/01_tcp_ar/107/00010772/s001_2013_09_26/00010772_s001_t000.edf 0
2898 data/edf/train/normal/01_tcp_ar/094/00009460/s001_2012_11_01/00009460_s001_t000.edf 0
2899 data/edf/train/abnormal/01_tcp_ar/066/00006687/s001_2010_06_02/00006687_s001_t000.edf 1
2900 data/edf/train/abnormal/01_tcp_ar/055/00005536/s003_2011_12_30/00005536_

2980 data/edf/train/normal/01_tcp_ar/103/00010363/s001_2013_05_30/00010363_s001_t000.edf 0
2981 data/edf/train/normal/01_tcp_ar/087/00008733/s001_2012_04_10/00008733_s001_t000.edf 0
2982 data/edf/train/normal/01_tcp_ar/096/00009610/s001_2012_11_12/00009610_s001_t000.edf 0
2983 data/edf/train/abnormal/01_tcp_ar/095/00009597/s001_2012_11_21/00009597_s001_t000.edf 1
2984 data/edf/train/normal/01_tcp_ar/094/00009456/s002_2012_10_09/00009456_s002_t000.edf 0
2985 data/edf/train/normal/01_tcp_ar/079/00007903/s001_2011_08_19/00007903_s001_t000.edf 0
2986 data/edf/train/abnormal/01_tcp_ar/062/00006273/s002_2011_03_07/00006273_s002_t001.edf 1
2987 data/edf/train/abnormal/01_tcp_ar/095/00009533/s002_2012_12_17/00009533_s002_t001.edf 1
2988 data/edf/train/abnormal/01_tcp_ar/075/00007574/s002_2011_10_13/00007574_s002_t001.edf 1
2989 data/edf/train/normal/01_tcp_ar/101/00010188/s001_2013_01_16/00010188_s001_t000.edf 0
2990 data/edf/train/normal/01_tcp_ar/071/00007132/s001_2010_11_05/00007132_s001_t0

In [28]:
# Save features and truth vector (y) sets to disk

feature_sets_file = 'final1_eeg_data'
np.savez_compressed(feature_sets_file, 
         x=x, 
         y=y)

In [29]:
# TEST: Load features

feature_sets_file = 'final1_eeg_data.npz'
feature_sets = np.load(feature_sets_file)
feature_sets.files

['x', 'y']